In [1]:
!unzip /content/drive/MyDrive/PP_yolo/data.zip

Archive:  /content/drive/MyDrive/PP_yolo/data.zip
   creating: data/log/
  inflating: data/log/events.out.tfevents.1624605012.9f8161edf78c.1420.7785.v2  
   creating: data/obj/
  inflating: data/obj/COCO_train2014_000000000036.jpg  
  inflating: data/obj/COCO_train2014_000000000036.txt  
  inflating: data/obj/COCO_train2014_000000000081.jpg  
  inflating: data/obj/COCO_train2014_000000000081.txt  
  inflating: data/obj/COCO_train2014_000000000086.jpg  
  inflating: data/obj/COCO_train2014_000000000086.txt  
  inflating: data/obj/COCO_train2014_000000000127.jpg  
  inflating: data/obj/COCO_train2014_000000000127.txt  
  inflating: data/obj/COCO_train2014_000000000247.jpg  
  inflating: data/obj/COCO_train2014_000000000247.txt  
  inflating: data/obj/COCO_train2014_000000000308.jpg  
  inflating: data/obj/COCO_train2014_000000000308.txt  
  inflating: data/obj/COCO_train2014_000000000315.jpg  
  inflating: data/obj/COCO_train2014_000000000315.txt  
  inflating: data/obj/COCO_train2014_00

In [6]:
import glob
import os

os.chdir('/content/data/obj')
myFiles = glob.glob('*.txt')
print(len(myFiles))

896


In [7]:
import shutil, os
files = myFiles
for f in files:
    shutil.copy(f, '/content/drive/MyDrive/PP_yolo/txt_annotation')

In [8]:
import glob
import os

os.chdir('/content/data/obj')
myFiles = glob.glob('*.jpg')
print(len(myFiles))

896


In [9]:
import shutil, os
files = myFiles
for f in files:
    shutil.copy(f, '/content/drive/MyDrive/PP_yolo/train_images')

In [10]:
import os
import cv2
 
# 
originLabelsDir = '/content/drive/MyDrive/PP_yolo/txt_annotation'
 # Converted file save path
saveDir = '/content/drive/MyDrive/PP_yolo/annos.txt'                                                                           
 # Picture path corresponding to the original label
originImagesDir = '/content/drive/MyDrive/PP_yolo/train_images'
 
txtFileList = os.listdir(originLabelsDir)
with open(saveDir, 'w') as fw:
    for txtFile in txtFileList:
        with open(os.path.join(originLabelsDir, txtFile), 'r') as fr:
            labelList = fr.readlines()
            for label in labelList:
                label = label.strip().split()
                x = float(label[1])
                y = float(label[2])
                w = float(label[3])
                h = float(label[4])
 
                # convert x,y,w,h to x1,y1,x2,y2
                imagePath = os.path.join(originImagesDir,
                                         txtFile.replace('txt', 'jpg'))
                image = cv2.imread(imagePath)
                H, W, _ = image.shape
                x1 = (x - w / 2) * W
                y1 = (y - h / 2) * H
                x2 = (x + w / 2) * W
                y2 = (y + h / 2) * H
                                 # In order to match the coco label method, the label serial number is calculated from 1
                fw.write(txtFile.replace('txt', 'jpg') + ' {} {} {} {} {}\n'.format(int(label[0]) + 1, x1, y1, x2, y2))
 
        print('{} done'.format(txtFile))

COCO_train2014_000000049402.txt done
COCO_train2014_000000115328.txt done
COCO_train2014_000000475533.txt done
COCO_train2014_000000377409.txt done
COCO_train2014_000000008339.txt done
COCO_train2014_000000123282.txt done
COCO_train2014_000000000542.txt done
COCO_train2014_000000418868.txt done
COCO_train2014_000000385589.txt done
COCO_train2014_000000303404.txt done
COCO_train2014_000000385239.txt done
COCO_train2014_000000147924.txt done
COCO_train2014_000000450716.txt done
COCO_train2014_000000205067.txt done
COCO_train2014_000000516344.txt done
COCO_train2014_000000008436.txt done
COCO_train2014_000000434379.txt done
COCO_train2014_000000065894.txt done
COCO_train2014_000000041434.txt done
COCO_train2014_000000573874.txt done
COCO_train2014_000000139970.txt done
COCO_train2014_000000188587.txt done
COCO_train2014_000000303108.txt done
COCO_train2014_000000106688.txt done
COCO_train2014_000000533050.txt done
COCO_train2014_000000016470.txt done
COCO_train2014_000000041442.txt done
C

In [15]:
! cp -r /content/drive/MyDrive/PP_yolo/annos.txt /content/drive/MyDrive/PP_yolo/train_images

In [16]:

import json
import os
import cv2
 
 # ------------ Use os to extract the image name in the images folder, and read the BBox into it ------------
 #root path, which contains images (picture folder), annos.txt (bbox annotation), classes.txt (category label),
 # and annotations folder (created automatically if not, used to save the last json)
root_path = '/content/drive/MyDrive/PP_yolo'
 # Used to create a training set or validation set
Phase = 'train' # need to be corrected
 
 # dataset is used to save image information and annotation information of all data
dataset = {'categories': [], 'annotations': [], 'images': []}
 
 # Open category label
with open(os.path.join(root_path, 'annos.txt')) as f:
    classes = f.read().strip().split()
 
 # Establish the correspondence between category labels and numeric ids
for i, cls in enumerate(classes, 1):
    dataset['categories'].append({'id': i, 'name': cls, 'supercategory': 'mark'})
 
 # Read the image name of the images folder
indexes = os.listdir(os.path.join(root_path, 'train_images'))
 
 # Statistics Processing the number of pictures
global count
count = 0
 
 # Read Bbox information
with open(os.path.join(root_path, 'annos.txt')) as tr:
    annos = tr.readlines()
 
         # --------------- Then, the above data is converted to the format required by COCO ---------------
    for k, index in enumerate(indexes):
        count += 1
                 # Read images with opencv to get the width and height of the image
        im = cv2.imread(os.path.join(root_path, 'train_images/') + index)
        height, width, _ = im.shape
 
                 # Add image information to the dataset
        dataset['images'].append({'file_name': index,
                                  'id': k,
                                  'width': width,
                                  'height': height})
 
        for ii, anno in enumerate(annos):
            parts = anno.strip().split()
 
                         # Add a tag if the name of the image and the name of the tag are on
            if parts[0] == index:
                                 # 
                cls_id = parts[1]
                # x_min
                x1 = float(parts[2])
                # y_min
                y1 = float(parts[3])
                # x_max
                x2 = float(parts[4])
                # y_max
                y2 = float(parts[5])
                width = max(0, x2 - x1)
                height = max(0, y2 - y1)
                dataset['annotations'].append({
                    'area': width * height,
                    'bbox': [x1, y1, width, height],
                    'category_id': int(cls_id),
                    'id': i,
                    'image_id': k,
                    'iscrowd': 0,
                                         # mask, the rectangle is the four vertices clockwise from the top left corner
                    'segmentation': [[x1, y1, x2, y1, x2, y2, x1, y2]]
                })
 
        print('{} images handled'.format(count))
 
 #Save the resulting folder
folder = os.path.join(root_path, 'annotations')
if not os.path.exists(folder):
  os.makedirs(folder)
json_name = os.path.join(root_path, 'annotations/img.json'.format(Phase))
with open(json_name, 'w') as f:
  json.dump(dataset, f)
 

1 images handled
2 images handled
3 images handled
4 images handled
5 images handled
6 images handled
7 images handled
8 images handled
9 images handled
10 images handled
11 images handled
12 images handled
13 images handled
14 images handled
15 images handled
16 images handled
17 images handled
18 images handled
19 images handled
20 images handled
21 images handled
22 images handled
23 images handled
24 images handled
25 images handled
26 images handled
27 images handled
28 images handled
29 images handled
30 images handled
31 images handled
32 images handled
33 images handled
34 images handled
35 images handled
36 images handled
37 images handled
38 images handled
39 images handled
40 images handled
41 images handled
42 images handled
43 images handled
44 images handled
45 images handled
46 images handled
47 images handled
48 images handled
49 images handled
50 images handled
51 images handled
52 images handled
53 images handled
54 images handled
55 images handled
56 images handled
5

In [17]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 14312, done.
remote: Counting objects: 100% (1258/1258), done.
remote: Compressing objects: 100% (646/646), done.
remote: Total 14312 (delta 752), reused 993 (delta 611), pack-reused 13054
Receiving objects: 100% (14312/14312), 133.61 MiB | 45.15 MiB/s, done.
Resolving deltas: 100% (10089/10089), done.


In [23]:
%cd /content/


/content


In [24]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 14312, done.
remote: Counting objects: 100% (1258/1258), done.
remote: Compressing objects: 100% (664/664), done.
remote: Total 14312 (delta 745), reused 981 (delta 593), pack-reused 13054
Receiving objects: 100% (14312/14312), 133.64 MiB | 45.64 MiB/s, done.
Resolving deltas: 100% (10074/10074), done.


In [25]:
pip install paddledet==2.1.0 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 419kB 214kB/s 
     |████████████████████████████████| 1.5MB 2.4MB/s 
     |████████████████████████████████| 153kB 11.5MB/s 
     |████████████████████████████████| 2.7MB 11.3MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 204kB 2.7MB/s 
     |████████████████████████████████| 194kB 20.6MB/s 
     |████████████████████████████████| 2.0MB 20.9MB/s 
     |████████████████████████████████| 51kB 1.8MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 1.9MB 14.7MB/s 
     |████████████████████████████████| 7.2MB 19.7MB/s 
     |████████████████████████████████| 102kB 7.4MB/s 
     |████████████████████████████████| 348kB 20.8MB/s 
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux